In [1]:
import numpy as np
import pandas as pd
import statsmodels as sm
import sklearn as skl
import math
import matplotlib as mpl

In [2]:
updated_cols = pd.read_excel("catan_data.xlsx").iloc[0].T

C:\Users\maxfu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
df = pd.read_excel("catan_data.xlsx")

C:\Users\maxfu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
renamedict = dict(zip(df.columns, list(updated_cols)))

In [5]:
df = df.rename(columns = renamedict).iloc[1:]

In [6]:
for col in df.columns:
    df[col] = pd.to_numeric(df[col],errors = "ignore")

In [7]:
df

,season,Session,game,loc,player,score,special,place,position,desert,...,prox_port_ore,prox_port_gen,t_sum_wood,t_sum_clay,t_sum_sheep,t_sum_grain,t_sum_ore,t_sum_paper,t_sum_coin,t_sum_fabric
1,2022,2022-03-19,1,Husum (Memeler Str.),PF,7,NaN,2,first,center,...,1,1,16,15,11,27,13,5,3,11
2,2022,2022-03-19,1,Husum (Memeler Str.),MF,14,NaN,1,second,center,...,0,1,16,15,11,27,13,5,3,11
3,2022,2022-03-19,1,Husum (Memeler Str.),JHC,5,NaN,3,third,center,...,0,0,16,15,11,27,13,5,3,11
4,2022,2022-03-19,2,Husum (Memeler Str.),JHC,7,NaN,2,first,outer,...,0,0,18,16,10,19,13,5,6,6
5,2022,2022-03-19,2,Husum (Memeler Str.),MF,13,NaN,1,second,outer,...,0,0,18,16,10,19,13,5,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2022,2022-11-19,24,Hamburg (Kegelhofstr.),JHC,10,3.0,2,second,outer,...,0,0,11,4,17,20,17,5,12,9
72,2022,2022-11-19,24,Hamburg (Kegelhofstr.),PF,13,7.0,1,third,outer,...,0,1,11,4,17,20,17,5,12,9
73,2022,2022-11-19,25,Hamburg (Kegelhofstr.),PF,9,2.0,2,first,second,...,0,0,15,24,15,10,7,15,0,5
74,2022,2022-11-19,25,Hamburg (Kegelhofstr.),JHC,13,4.0,1,second,second,...,0,0,15,24,15,10,7,15,0,5


In [16]:
pd.DataFrame(df.groupby("place").agg("sum")["sum"])/(len(df)/3)

,sum
place,
1,31.28
2,31.40
3,31.12


In [25]:
[col for col in df]

['season',
 'Session',
 'game',
 'loc',
 'player',
 'score',
 'special',
 'place',
 'position',
 'desert',
 'sum',
 'count_num',
 'count_res',
 'p_sum_wood',
 'p_sum_clay',
 'p_sum_sheep',
 'p_sum_grain',
 'p_sum_ore',
 'p_sum_paper',
 'p_sum_coin',
 'p_sum_fabric',
 'port_wood',
 'port_clay',
 'port_sheep',
 'port_grain',
 'port_ore',
 'port_gen',
 'prox_port_wood',
 'prox_port_clay',
 'prox_port_sheep',
 'prox_port_grain',
 'prox_port_ore',
 'prox_port_gen',
 't_sum_wood',
 't_sum_clay',
 't_sum_sheep',
 't_sum_grain',
 't_sum_ore',
 't_sum_paper',
 't_sum_coin',
 't_sum_fabric',
 'hafen']

In [23]:
df["hafen"] = 0

df.loc[(df["port_wood"]==1) | (df["port_clay"]==1) | (df["port_sheep"]==1) | 
       (df["port_grain"]==1 )| (df["port_ore"]==1) | (df["port_gen"]==1),"hafen"] = 1

In [37]:
for col in [col for col in df if "p_sum" in col]:
    print(col[6:]+":",df[col].mean())

wood: 5.28
clay: 4.6
sheep: 4.386666666666667
grain: 5.84
ore: 3.6666666666666665
paper: 3.6533333333333333
coin: 1.2933333333333332
fabric: 2.546666666666667
